In [ ]:
# Import Important Library
import pandas as pd

In [ ]:
# Define Dataframe
df_environmental = pd.read_excel('Environmental_Dataset.xlsx')
df_financial = pd.read_excel('Financial_Dataset.xlsx')
df_social = pd.read_excel('Social_Dataset.xlsx')
df_economic = pd.read_excel('Economic_Dataset.xlsx')
df_geospatial = pd.read_excel('Geospatial_Dataset.xlsx')
df_environmental_financial = pd.merge(df_environmental, df_financial, on='Project_ID', how='inner')
df_economic_social = pd.merge(df_economic, df_social, on='Project_ID', how='inner')
df_geospatial_financial = pd.merge(df_geospatial, df_financial, on='Project_ID', how='inner')

#### Question 1: Conditional Statements (If-Else) and Arithmetic Operations

In [ ]:
for idx, row in df_environmental_financial.iterrows():
  if str(row['Project_ID']).startswith('PLTS'):
      ratio = row['CO2_Reduction']/(row['Investment_Cost'] * 1_000_000)
      if ratio >= 0.5:
        desc = 'High'
      else:
        desc = 'Low'
      print(f'{row["Project_ID"]}: {ratio} ({desc}) ')

PLTS-NTT-001: 0.0005 (Low) 
PLTS-JATIM-001: 0.0004494830944413924 (Low) 
PLTS-SULS-001: 0.00047808764940239046 (Low) 
PLTS-NTB-001: 0.00044444444444444447 (Low) 
PLTS-JABW-001: 0.0004318181818181818 (Low) 


#### Question 2: For Loop and Lists

In [ ]:
sum_co2_reduction = 0
count = 0
for idx, row in df_environmental.iterrows():
  if str(row['Project_ID']).startswith('PLTM'):
    sum_co2_reduction += row['CO2_Reduction']
    count += 1
avg_co2_reduction = sum_co2_reduction / count
print(f'Average CO2 Reduction for PLTM projects: {avg_co2_reduction}')

Average CO2 Reduction for PLTM projects: 34600.0


#### Question 3: While Loop and User Input

In [ ]:
while True:
  project_id = input('Enter Project_ID (or `Done` to finish): ')
  if project_id == 'Done':
    break
  if project_id in df_social['Project_ID'].values:
    row = df_social[df_social['Project_ID'] == project_id]
    print(f'Project Name: {row["Project_ID"].values[0]} - Land Status: {row["Land_Status"].values[0]}, Tingkat Konflik: {row["Tingkat_Konflik"].values[0]}')
  else:
    print('Project Not Found')



Enter Project_ID (or `Done` to finish): Done


#### Question 4: Dictionary and Conditional Filtering

In [ ]:
# Columns to clean
columns_to_clean = ['Daya_Tarik_Investasi', 'Tingkat_Konflik']

# Apply split and strip to each target column
for col in columns_to_clean:
    df_economic_social[col] = df_economic_social[col].apply(lambda x: x.split(':')[0].strip())

In [ ]:
project_dict = {}
for _, row in df_economic_social.iterrows():
  if row['Daya_Tarik_Investasi'] == "High" and row["Tingkat_Konflik"] == "Low":
    project_dict[row['Project_ID']] = (row['Daya_Tarik_Investasi'], row['Tingkat_Konflik'])

print('Projects with High Investment Attractiveness and Low Conflict: ')
for project_id in project_dict:
  print(project_id)

Projects with High Investment Attractiveness and Low Conflict: 
PLTM-SUMUT-001
PLTS-JATIM-001
PLTS-NTB-001
PLTS-JABW-001


#### Question 5: Functions and Arithmetic

In [ ]:
df_geospatial_financial['Efisiensi_Lokasi'] = df_geospatial_financial['Efisiensi_Lokasi'].apply(lambda x: x.split(':')[0].strip())

In [ ]:
def calculate_total_investment(data):
  total_investment = 0
  for _, row in data.iterrows():
    if row['Efisiensi_Lokasi'] == 'High':
      total_investment += row['Investment_Cost']
  return total_investment

print(f'Total Investment for High Efficiency Locations: {calculate_total_investment(df_geospatial_financial)}')

Total Investment for High Efficiency Locations: 955.73


#### Question 7: Error Handling in Loops

In [ ]:
project_ids = ['PLTS-JATIM-001', 'PLTS-NTB-001', 'PLTS-SULSEL-003']


total_energy_output = 0
valid_projects = 0

for project_id in project_ids:
    try:

        energy_output = df_environmental.loc[df_environmental['Project_ID'] == project_id, 'Energy_Output'].values[0]
        total_energy_output += energy_output
        valid_projects += 1
    except IndexError:
        print(f"Project_ID '{project_id}' not found in the dataset.")
    except KeyError as e:
        print(f"Missing column in the dataset: {e}")

if valid_projects > 0:
    average_output = total_energy_output / valid_projects
    print(f"\nTotal Valid Projects: {valid_projects}")
    print(f"Average Energy Output: {average_output}")
else:
    print("No valid projects found.")


Project_ID 'PLTS-SULSEL-003' not found in the dataset.

Total Valid Projects: 2
Average Energy Output: 29000.0


#### Question Bonus

In [ ]:
df_economic_environmental_financial = pd.merge(df_environmental_financial, df_economic, on='Project_ID', how='inner')

In [ ]:
df_economic_environmental_financial.to_excel('question_bonus_dataset.xlsx', index=False)

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import warnings
warnings.filterwarnings('ignore')

# Loading dataset
data = pd.read_csv('question_bonus_dataset.csv')

# Checking for missing values and handling them
print("Missing values:\n", data.isnull().sum())
data = data.dropna()  # Drop rows with missing values if any

# Encoding categorical columns
categorical_cols = ['Konteks_Lingkungan', 'Peringkat_Dampak', 'Konteks_Proyek',
                    'Status_Rank', 'Konteks_Ekonomi', 'Daya_Tarik_Investasi']
label_encoders = {}
for col in categorical_cols:
    le = LabelEncoder()
    data[col] = le.fit_transform(data[col])
    label_encoders[col] = le

# Separating features and target
X = data.drop(['Project_ID', 'Daya_Tarik_Investasi'], axis=1)  # Drop ID and target
y = data['Daya_Tarik_Investasi']

# Scaling numerical features
numerical_cols = ['CO2_Reduction', 'Energy_Output', 'Environmental_Risk_Index',
                  'Investment_Cost', 'Revenue_Stream', 'Debt_Ratio',
                  'Payment_Delay', 'GDP_Growth', 'Interest_Rate', 'Bond_Yield']
scaler = StandardScaler()
X[numerical_cols] = scaler.fit_transform(X[numerical_cols])

# Splitting dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Defining Decision Tree Classifier
dt = DecisionTreeClassifier(random_state=42)

# Defining hyperparameter grid for tuning
param_grid = {
    'criterion': ['gini', 'entropy'],
    'max_depth': [5, 10, 15, 20, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Performing GridSearchCV for hyperparameter tuning
grid_search = GridSearchCV(estimator=dt, param_grid=param_grid,
                           cv=5, n_jobs=-1, verbose=2, scoring='accuracy')
grid_search.fit(X_train, y_train)

# Getting the best model
best_dt = grid_search.best_estimator_
print("Best parameters:", grid_search.best_params_)

# Predicting on test set
y_pred = best_dt.predict(X_test)

# Evaluating the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.4f}")
print("\nClassification Report:\n", classification_report(y_test, y_pred,
      target_names=label_encoders['Daya_Tarik_Investasi'].classes_))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))

# Feature importance
feature_importance = pd.DataFrame({
    'Feature': X.columns,
    'Importance': best_dt.feature_importances_
}).sort_values(by='Importance', ascending=False)
print("\nFeature Importance:\n", feature_importance)

Missing values:
 Project_ID                  0
CO2_Reduction               0
Energy_Output               0
Environmental_Risk_Index    0
Konteks_Lingkungan          0
Peringkat_Dampak            0
Investment_Cost             0
Revenue_Stream              0
Debt_Ratio                  0
Payment_Delay               0
Konteks_Proyek              0
Status_Rank                 0
GDP_Growth                  0
Interest_Rate               0
Bond_Yield                  0
Konteks_Ekonomi             0
Daya_Tarik_Investasi        0
dtype: int64
Fitting 5 folds for each of 90 candidates, totalling 450 fits
Best parameters: {'criterion': 'entropy', 'max_depth': 5, 'min_samples_leaf': 1, 'min_samples_split': 10}
Accuracy: 0.3650

Classification Report:
               precision    recall  f1-score   support

  High: 💵💵💵💵       0.08      0.02      0.03        64
 High: 💵💵💵💵💵       0.00      0.00      0.00        35
     Low: 💵💵       0.00      0.00      0.00        24
 Medium: 💵💵💵       0.39      0.94